In [ ]:
import pickle
import numpy as np
with open ('train_qa.txt','rb') as fp:
    train_data = pickle.load(fp)
train_data
with open ('test_qa.txt','rb') as fo:
    test_data = pickle.load(fo)
test_data

len(train_data)
len(test_data)
vocab= set()

all_data = test_data + train_data
for story,question,answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))
    
vocab.add('yes')
vocab.add('no')

max_story_len =  max([len(data[0])for data in all_data])
max_ques_len =  max([len(data[1])for data in all_data])

max_story_len

from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(filters = [])from keras.preprocessing.text import Tokenizer 
max_story_len

tokenizer.word_index
train_story_text =[]
train_question_text =[]
train_answers_text =[]

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
train_story_seq = tokenizer.texts_to_sequences(train_story_text)
train_story_seq 
def vectorize_stories(data , word_index=tokenizer.word_index , max_story_len = max_story_len , max_ques_len = max_ques_len ):
    X = []
    Xq = []
    Y = []
    for story,ques,ans in data :
        x = [word_index[word.lower()] for word in story]
        fg = [word_index[word.lower()] for word in ques]
        y = np.zeros(len(word_index)+1)
        y[word_index[ans]] = 1
        
        X.append(x)
        Xq.append(fg)
        Y.append(y)
    return(pad_sequences(X,max_story_len),
           pad_sequences(Xq,max_ques_len),
       
           np.array(Y))    
inputs_train,queries_train,answers_train = vectorize_stories(train_data)
vocab_len = len(vocab)+1
inputs_test,queries_test,answers_test = vectorize_stories(test_data)

from tensorflow.keras.models import Sequential ,Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate,LSTM
input_sequence = Input((max_story_len,))
question = Input((max_ques_len,))
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = vocab_len, output_dim = 64))
input_encoder_m.add(Dropout(0.3))



input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = vocab_len, output_dim = max_ques_len))
input_encoder_c.add(Dropout(0.3))
ques_encoder = Sequential()
ques_encoder.add(Embedding(input_dim = vocab_len, output_dim = 64,input_length = max_ques_len))
ques_encoder.add(Dropout(0.3))
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
ques_encoded = ques_encoder(question)


match = dot([input_encoded_m,ques_encoded],axes = (2,2))
match = Activation('softmax')(match)
response =  add([match,input_encoded_c])
response = Permute((2,1))(response)
answer = concatenate([response,ques_encoded])
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)

answer = Dense(vocab_len)(answer) 
answer = Activation('softmax')(answer)
model = Model([input_sequence,question],answer)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()
history = model.fit([inputs_train,queries_train],answers_train,
                    batch_size = 30, epochs = 22,
                    validation_data=([inputs_test,queries_test],answers_test)
                    )
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
model.save("chatbot_model")
model.load_weights("chatbot_model")
pred_results = model.predict(([inputs_test,queries_test]))
pred_results

test_data[90][0]
story = ' '.join(word for word in test_data[90][0])
story
queries = ' '.join(word for word in test_data[90][1])
queries
test_data[90][2]
val_max=np.argmax(pred_results[37])
for key,val in tokenizer.word_index.items():
    if val==val_max:
        k=key
print("predicted answer is ",k)
print("probability of certainity",pred_results[37][val_max])
